## Открытие и изучение файла

Импортируем необходимые для работы библиотеки: 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

Откроем файл с данными и выведем первые 10 строк на экран:

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Проверим данные:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.duplicated().sum()

0

Из полученного результата следует, что пропусков и дубликатов в данных нет, все данные типа, соответствующего им  

## Разбивка данных на выборки

Разделим исходные данные на обучающую выборку (`features_train` - признаки выборки, `target_train` - целевой признак), валидационную (`features_valid` - признаки выборки, `target_valid` - целевой признак) и тестовую (`features_test` - признаки выборки, `target_test` - целевой признак). Разделим в соотношении 3:1:1, т.е. у тестовой и валидационной выборок - по 20% от общей, у обучающей - 60%: 

Определим количество процентное соотношение единиц к общему количеству:

In [5]:
df.query('is_ultra == 1')['is_ultra'].count() / df.shape[0]

0.30647168637212197

Произведем разделение датасета таким образом, чтобы такое процентное соотношение сохранялось в выборках, а также сделаем проверку:

In [6]:
features = df.drop(['is_ultra'], axis = 1)
target = df['is_ultra']

df_train = df.copy()
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.2, random_state = 12345, stratify = target)
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size = 0.25, 
                                                                              random_state = 12345, stratify = target_train)

print(target_train[target_train == 1].count() / len(target_train))
print(target_test[target_test == 1].count() / len(target_test))
target_valid[target_valid == 1].count() / len(target_valid)

0.3065352697095436
0.30637636080870917


0.30637636080870917

## Исследование модели

Обучим модель на разных алгоритмах

Для начала проведем обучение на алгоритме "дерево решение". Будем менять параметр `max_depth` и определим качество данной модели (наибольшее значение `accuracy`), а также создадим список `accuracy_decision_tree_classifier_list`, содержащий значения `accuracy`, а также определим значение `max_depth`, при котором достигается максимальное значение `accuracy` на обучающей выборке при данном алгоритме (значение `max_depth` на 1 больше порядка в списке)::  

In [7]:
result_model_decision_tree_classifier = 0
accuracy_decision_tree_classifier_list = []

for depth in range(1, len(target_test)):
         
    model_decision_tree_classifier = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    model_decision_tree_classifier.fit(features_train, target_train)
    answer_decision_tree_classifier = model_decision_tree_classifier.predict(features_valid)
    accuracy_decision_tree_classifier = accuracy_score(answer_decision_tree_classifier, target_valid)
    accuracy_decision_tree_classifier_list.append(accuracy_decision_tree_classifier)
    
    
    if result_model_decision_tree_classifier < accuracy_decision_tree_classifier:
        result_model_decision_tree_classifier = accuracy_decision_tree_classifier       
          
print('Наибольшее значение accuracy: ', result_model_decision_tree_classifier)

for index in range(len(accuracy_decision_tree_classifier_list)):
    if accuracy_decision_tree_classifier_list[index] == result_model_decision_tree_classifier:
        print('Лучшее значение гиперпараметра max_depth: ', index + 1)

Наибольшее значение accuracy:  0.8164852255054432
Лучшее значение гиперпараметра max_depth:  5


Проведем обучение на алгоритме "логистическая регрессия". Будем менять параметр `max_iter` и определим качество данной модели (наибольшее значение `accuracy`), а также создадим список `accuracy_model_LogisticRegression_list`, содержащий значения `accuracy`:  

In [8]:
result_model_LogisticRegression = 0
accuracy_model_LogisticRegression_list = []

for iteration in range(100, 100000, 100):
    
    model_LogisticRegression = LogisticRegression(random_state = 12345, solver = 'lbfgs', max_iter = iteration)
    model_LogisticRegression.fit(features_train, target_train)
    answer_LogisticRegression = model_LogisticRegression.predict(features_valid)
    accuracy_model_LogisticRegression = accuracy_score(answer_LogisticRegression, target_valid)
    accuracy_model_LogisticRegression_list.append(accuracy_model_LogisticRegression)
    
    if result_model_LogisticRegression < accuracy_model_LogisticRegression:
        result_model_LogisticRegression = accuracy_model_LogisticRegression

print(result_model_LogisticRegression)

0.7558320373250389


Определим значение `max_iter`, при котором достигается максимальное значение `accuracy` на обучающей выборке при данном алгоритме:

In [9]:
for index in range(len(accuracy_model_LogisticRegression_list)):
    if accuracy_model_LogisticRegression_list[index] == result_model_LogisticRegression:
        print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

Из полученного результата следует, что качество модели не зависит от количества итераций, параметр `max_iter` можем не учитывать при обучении модели.

Проведем обучение на алгоритме "случайный лес". Будем менять параметр `n_estimators` и определим качество данной модели (наибольшее значение `accuracy`), а также создадим список `accuracy_model_RandomForestClassifier_list`, содержащий значения `accuracy`:  

In [10]:
result_RandomForestClassifier = 0
accuracy_model_RandomForestClassifier_list = []

for est in range(1, 100):
    
    model_RandomForestClassifier = RandomForestClassifier(random_state = 12345, n_estimators = est)
    model_RandomForestClassifier.fit(features_train, target_train)
    answer_RandomForestClassifier = model_RandomForestClassifier.predict(features_valid)
    accuracy_model_RandomForestClassifier = accuracy_score(answer_RandomForestClassifier, target_valid)
    accuracy_model_RandomForestClassifier_list.append(accuracy_model_RandomForestClassifier)
    
    if result_RandomForestClassifier < accuracy_model_RandomForestClassifier:
        result_RandomForestClassifier = accuracy_model_RandomForestClassifier
    
result_RandomForestClassifier

0.8195956454121306

Определим значение `n_estimators`, при котором достигается максимальное значение `accuracy` на обучающей выборке при данном алгоритме (значение `n_estimators` на 1 больше порядка в списке):

In [11]:
for index in range(len(accuracy_model_RandomForestClassifier_list)):
    if accuracy_model_RandomForestClassifier_list[index] == result_RandomForestClassifier:
        print(index + 1)

24
42


Значения `n_estimators`, на которых достигается максимальное значение `accuracy` - 24 и 42

Таким образом, наибольшее значение `accuracy` на обучающей выборке - 0.8196 было достигнуто на модели "случайный лес", наименьшее значение - 0.7558 - на "логистической регресии", среднее значение - 0.8165 - на "дереве решений".

## Проверка модели на тестовой выборке

В пункте 3 были определены на разных алгоритмах максимальные значения `accuracy`, а также гиперпараметры, соответствующие им. Теперь проверим модель с наибольшим значением `accuracy` - "случайный лес" - на тестовой выборке

Для модели "случайный лес" наилучшие значения гиперпараметра `случайном лесе` были определены 24 и 42. Сделаем проверку:

In [12]:
model_RandomForestClassifier_test = RandomForestClassifier(random_state = 12345, n_estimators = 24)
model_RandomForestClassifier_test.fit(features_train, target_train)
answer_RandomForestClassifier_test = model_RandomForestClassifier_test.predict(features_test)
accuracy_model_RandomForestClassifier_test = accuracy_score(answer_RandomForestClassifier_test, target_test)
accuracy_model_RandomForestClassifier_test

0.8009331259720062

In [13]:
model_RandomForestClassifier_test = RandomForestClassifier(random_state = 12345, n_estimators = 42)
model_RandomForestClassifier_test.fit(features_train, target_train)
answer_RandomForestClassifier_test = model_RandomForestClassifier_test.predict(features_test)
accuracy_model_RandomForestClassifier_test = accuracy_score(answer_RandomForestClassifier_test, target_test)
accuracy_model_RandomForestClassifier_test

0.8087091757387247

Качество модели снизилось менее, чем на 1%, что незначительно

Таким образом можно сделать вывод, что качество модели "случайный лес" на тестовой выборке снизилось незначительно (с 0.8165 до 0.8087), при этом значение `accuracy` прошло минимальный качества порог 0.75. Полученная модель устойчива на выборках и дает результат с поставленной точностью.